Initializing pyspark session

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import types as T

Defining function for calculating capture rate and conversion rate

In [ ]:
def get_cap_conv_rate(df,target_cols,long_target)
    list1=target_cols
    list3 = []
    list4 = []
    list5 = []
    final_target = long_target

    for flag in list1
        conversion_rate = (df.filter((F.col(flag)==1) & (F.col(final_target)  ==1)).count())/(df.filter(F.col(flag)==1).count())
        list3.append(conversion_rate)
        bad_count = (df.filter(F.col(flag)==1).count())
        total_count = (df.count())
        list4.append(bad_count)
    for flag in list1
        capture_rate = (df.filter((F.col(flag)==1) & (F.col(final_target)==1)).count()) / (df.filter(F.col(final_target)==1).count())
        list5.append(capture_rate)
    data = list(zip(list1,list4,[total_count]*len(list1),list5,list3))
    deptSchema = StructType([
        StructField('flag',StringType(),True),
        StructField('bad_count',IntegerType(),True),
        StructField('total_count',IntegerType(),True),
        StructField('Capture_rate',DoubleType(),True),
        StructField('Conversion_rate',DoubleType(),True)])
    df_final = spark.createDataFrame(data, schema=deptSchema)
    return df_final


Creating target columns: assuming the data has dpd data for all the dpd months

In [ ]:
dpd_months = [6,9,12,15,18,24]

exp2 = ['*']\
+[f"case when max_dpd{i}mob >= 30 then 1 else 0 end as targer_{i}mob_30_flag" for i in dpd_months]\
+[f"case when max_dpd{i}mob >= 60 then 1 else 0 end as targer_{i}mob_60_flag" for i in dpd_months]\
+[f"case when max_dpd{i}mob >= 90 then 1 else 0 end as targer_{i}mob_90_flag" for i in dpd_months]\
+[f"case when max_dpd{i}mob >= 120 then 1 else 0 end as targer_{i}mob_120_flag" for i in dpd_months]\
+[f"case when max_dpd{i}mob >= 150 then 1 else 0 end as targer_{i}mob_150_flag" for i in dpd_months]\
+[f"case when max_dpd{i}mob >= 180 then 1 else 0 end as targer_{i}mob_180_flag" for i in dpd_months]


Running Cap Con analysis

In [ ]:
df = spark.sql("select * from data_table")

df1 = df.selectExpr(exp2)
long_target = "target_24mob_180_flag"
target_cols = [i for i in df1.columns if 'target' in 1]
df_final = get_cap_conv_rate(df1,target_cols,long_target)